### In this notebook I am getting data from [CPJ](https://cpj.org/data/) on the killings of Journalists from 1992 to 2024.

In [12]:
#import the necessary module
import pandas as pd

In [13]:
# What did I do to get here:
# 1. I went onto the database, right-clicked and selected "inspect elements"
# 2. I went onto "Network" and I copied the undocumented API as bash
# 3. I went onto curl converter and pasted what I had copied.
# 4. The result below is what I got from Curl converter
import requests

headers = {
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-language': 'el-GR,el;q=0.9,en;q=0.8',
    'origin': 'https://cpj.org',
    'priority': 'u=1, i',
    'referer': 'https://cpj.org/',
    'sec-ch-ua': '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
}

response = requests.get(
    'https://datamanager.cpj.org/api/datamanager/reports/entries?distinct(personId)&includes=organizations,fullName,location,status,typeOfDeath,charges,startDisplay,mtpage,country,type,motiveConfirmed&sort=-fullName&pageNum=1&pageSize=20&in(status,%27Killed%27)&or(eq(type,%22media%20worker%22),in(motiveConfirmed,%27Confirmed%27,%27Unconfirmed%27))&in(type,%27Journalist%27,%27Media%20Worker%27)&ge(year,1993)&le(year,2024)',
    headers=headers,
)

In [3]:
#If response == 200 it means we are good to go
response

<Response [200]>

In [4]:
# saving the response as data
data = response.json()
data

{'rowCount': 2274,
 'pageNum': 1,
 'pageSize': '20',
 'pageCount': 114,
 'data': [{'organizations': 'Todzhikiston',
   'fullName': 'Zukhuruddin Suyari',
   'location': 'Kurgan-Tiube',
   'status': 'Killed',
   'typeOfDeath': 'Murder',
   'charges': None,
   'startDisplay': 'March 1, 1993',
   'mtpage': 'https://cpj.org/data/people/zukhuruddin-suyari/',
   'country': 'Tajikistan',
   'type': 'Journalist',
   'motiveConfirmed': 'Confirmed'},
  {'organizations': 'Mitra TV',
   'fullName': 'Zubair Hatami',
   'location': 'Kabul',
   'status': 'Killed',
   'typeOfDeath': 'Dangerous Assignment',
   'charges': None,
   'startDisplay': 'December 21, 2014',
   'mtpage': 'https://cpj.org/data/people/zubair-hatami/',
   'country': 'Afghanistan',
   'type': 'Journalist',
   'motiveConfirmed': 'Confirmed'},
  {'organizations': 'Jang',
   'fullName': 'Zubair Ahmed Mujahid',
   'location': 'Mirpur Khas',
   'status': 'Killed',
   'typeOfDeath': 'Murder',
   'charges': None,
   'startDisplay': 'Novemb

### The steps above are for one seperate page. Below we will do that for all pages

In [8]:
# the first part is the same as above
import requests

headers = {
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-language': 'el-GR,el;q=0.9,en;q=0.8',
    'origin': 'https://cpj.org',
    'priority': 'u=1, i',
    'referer': 'https://cpj.org/',
    'sec-ch-ua': '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
}
# here we set the url as base_url
base_url = 'https://datamanager.cpj.org/api/datamanager/reports/entries'
#and create the params based on what the first url we used has. For example, in the second cell we see that the url is:
#https://datamanager.cpj.org/api/datamanager/reports/entries?distinct(personId)&includes=organizations,fullName,location,status,typeOfDeath,charges,startDisplay,mtpage,country,type,motiveConfirmed&sort=-fullName&pageNum=1&pageSize=20&in(status,%27Killed%27)&or(eq(type,%22media%20worker%22),in(motiveConfirmed,%27Confirmed%27,%27Unconfirmed%27))&in(type,%27Journalist%27,%27Media%20Worker%27)&ge(year,1993)&le(year,2024)',
# the first part is the base url (as mentioned)
#then all params are what follows
params = {
    'distinct': 'personId',
    'includes': 'organizations,fullName,location,status,typeOfDeath,charges,startDisplay,mtpage,country,type,motiveConfirmed',
    'sort': '-fullName',
    'pageNum': 1,
    'pageSize': 20,
    'in': 'status,\'Killed\'',
    'or': 'eq(type,"media worker"),in(motiveConfirmed,\'Confirmed\',\'Unconfirmed\')',
    'in': 'type,\'Journalist\',\'Media Worker\'',
    'ge': 'year,1992',
    'le': 'year,2024'
}
#we create an empty list to store the results
all_results = []
# we need a while loop because we don't know for how many pages is this needed
while True:
    # and we do the same as before, but now for all pages
    response = requests.get(base_url, headers=headers, params=params)
    data = response.json()
    
    # appending the current page of results to the all_results list
    all_results.extend(data.get('data', []))
    
    # checking to see if there are more pages
    current_page = data.get('pageNum', 1)
    total_pages = data.get('pageCount', 1)
    
    if current_page >= total_pages:
        break
    
    # updating the page number to get the next page of results
    params['pageNum'] += 1

# here i print all the results to see if it worked
print(all_results)


[{'organizations': 'Todzhikiston', 'fullName': 'Zukhuruddin Suyari', 'location': 'Kurgan-Tiube', 'status': 'Killed', 'typeOfDeath': 'Murder', 'charges': None, 'startDisplay': 'March 1, 1993', 'mtpage': 'https://cpj.org/data/people/zukhuruddin-suyari/', 'country': 'Tajikistan', 'type': 'Journalist', 'motiveConfirmed': 'Confirmed'}, {'organizations': 'Mitra TV', 'fullName': 'Zubair Hatami', 'location': 'Kabul', 'status': 'Killed', 'typeOfDeath': 'Dangerous Assignment', 'charges': None, 'startDisplay': 'December 21, 2014', 'mtpage': 'https://cpj.org/data/people/zubair-hatami/', 'country': 'Afghanistan', 'type': 'Journalist', 'motiveConfirmed': 'Confirmed'}, {'organizations': 'Jang', 'fullName': 'Zubair Ahmed Mujahid', 'location': 'Mirpur Khas', 'status': 'Killed', 'typeOfDeath': 'Murder', 'charges': None, 'startDisplay': 'November 23, 2007', 'mtpage': 'https://cpj.org/data/people/zubair-ahmed-mujahid/', 'country': 'Pakistan', 'type': 'Journalist', 'motiveConfirmed': 'Confirmed'}, {'organi

In [9]:
# i have a list of dictionaries so I turn it into a dataframe
df = pd.DataFrame(all_results)
df

,organizations,fullName,location,status,typeOfDeath,charges,startDisplay,mtpage,country,type,motiveConfirmed
0,Todzhikiston,Zukhuruddin Suyari,Kurgan-Tiube,Killed,Murder,None,"March 1, 1993",https://cpj.org/data/people/zukhuruddin-suyari/,Tajikistan,Journalist,Confirmed
1,Mitra TV,Zubair Hatami,Kabul,Killed,Dangerous Assignment,None,"December 21, 2014",https://cpj.org/data/people/zubair-hatami/,Afghanistan,Journalist,Confirmed
2,Jang,Zubair Ahmed Mujahid,Mirpur Khas,Killed,Murder,None,"November 23, 2007",https://cpj.org/data/people/zubair-ahmed-mujahid/,Pakistan,Journalist,Confirmed
3,Freelance,Zoreslav Zamoysky,Bucha,Killed,Dangerous Assignment,None,"March 5 - March 15, 2022",https://cpj.org/data/people/zoreslav-zamoysky/,Ukraine,Journalist,Confirmed
4,Srpsko Slovo,Zivko Filipovic,Ilidza,Killed,Unknown,None,"January 30, 1993",https://cpj.org/data/people/zivko-filipovic/,Bosnia,Journalist,Unconfirmed
...,...,...,...,...,...,...,...,...,...,...,...
2326,"Radio Azadi,Radio Free Europe/Radio Liberty",Abadullah Hananzai,Kabul,Killed,Murder,None,"April 30, 2018",https://cpj.org/data/people/abadullah-hananzai/,Afghanistan,Journalist,Confirmed
2327,Al-Hurra,Unidentified,Iskandiriyah,Killed,Unknown,None,"February 25, 2005",https://cpj.org/data/people/unidentified-2/,Iraq,Media Worker,Confirmed
2328,Iraq Media Network,Sabah Salman,Baghdad,Killed,Unknown,None,"February 7, 2007",https://cpj.org/data/people/sabah-salman/,Iraq,Media Worker,Confirmed
2329,Al-Watan,Khaled Mohammad Nofan,an area southwest of Kirkuk,Killed,Unknown,None,"October 14, 2007",https://cpj.org/data/people/khaled-mohammad-no...,Iraq,Media Worker,Confirmed


In [11]:
#I save my results as a .csv and .xlsx files
df.to_csv("CPJ_killings.csv")
df.to_excel("CPJ_killings.xlsx")